<a href="https://colab.research.google.com/github/dawikrol/Engineering-thesis/blob/main/Scripts/Wind_potential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Wind potential**





### Lib import

In [2]:
import pandas as pd
from google.colab import files
import io
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### Uploading data

You find data in project's GitHub repository in Data/Wind.

Data regarding atmospheric conditions come from: https://ec.europa.eu/jrc/en/pvgis



In [3]:
uploaded = files.upload()

Saving Limanowski_01_W.csv to Limanowski_01_W.csv
Saving Nowosądecki_01_W.csv to Nowosądecki_01_W.csv
Saving Nowotarski_01_W.csv to Nowotarski_01_W.csv
Saving NowySącz_01_W.csv to NowySącz_01_W.csv
Saving Tatrzański_01_W.csv to Tatrzański_01_W.csv


### Creating raw dataframes

In [4]:
dataframes = {}

for file in uploaded:
    name, region, kind  = file.split('_')
    dataframes[name] = pd.read_csv(io.BytesIO(uploaded[file]), error_bad_lines=False,
                                   index_col=None, header=None, sep='\n')

In [ ]:
dataframes[list(dataframes.keys())[0]].size

8787

In [ ]:
dataframes[list(dataframes.keys())[0]].head(30)

,0
0,Latitude (decimal degrees): 49.723
1,Longitude (decimal degrees): 20.395
2,Elevation (m): 400.0
3,"month,year"
4,"1,2006"
5,"2,2010"
6,"3,2006"
7,"4,2012"
8,"5,2012"
9,"6,2011"


In [ ]:
dataframes[list(dataframes.keys())[0]].tail(30)

,0
8757,"20101231:0400,-7.55,69.97,0.0,-0.0,0.0,220.02,..."
8758,"20101231:0500,-7.22,70.29,0.0,-0.0,0.0,223.4,3..."
8759,"20101231:0600,-6.39,70.01,0.0,-0.0,0.0,233.68,..."
8760,"20101231:0700,-5.57,69.74,3.0,0.0,3.0,243.97,3..."
8761,"20101231:0800,-4.75,69.46,67.0,23.3,63.0,254.2..."
8762,"20101231:0900,-4.11,70.33,109.0,28.05,102.0,26..."
8763,"20101231:1000,-3.46,71.2,140.0,44.83,127.0,266..."
8764,"20101231:1100,-2.82,72.07,136.0,34.45,126.0,27..."
8765,"20101231:1200,-3.06,76.09,147.0,119.81,117.0,2..."
8766,"20101231:1300,-3.31,80.12,74.0,0.0,74.0,275.47..."


### Creating list of shortcuts

In [5]:
info_df = dataframes[list(dataframes.keys())[0]].iloc[8777:8786]
info_df.columns = ['defu']
info_df = info_df.defu.str.split(':', expand=True)
info_df.columns = ['symbol', 'meaning']
info_df.set_index('symbol', inplace=True)
info_df


,meaning
symbol,
T2m,2-m air temperature (degree Celsius)
RH,relative humidity (%)
G(h),Global irradiance on the horizontal plane (W/m2)
Gb(n),Beam/direct irradiance on a plane always norm...
Gd(h),Diffuse irradiance on the horizontal plane (W...
IR(h),Surface infrared (thermal) irradiance on a ho...
WS10m,10-m total wind speed (m/s)
WD10m,"10-m wind direction (0 = N, 90 = E) (degree)"
SP,Surface (air) pressure (Pa)


### Data cleaning (1)

In [110]:
cleared_df = {}

for key in dataframes:
    cl_data = dataframes[key].iloc[16:8777]
    cl_data.columns = ['defu'] #the default column name is inconvenient
    cl_data = cl_data.defu.str.split(',', expand=True)
    cl_data.columns = cl_data.iloc[0]
    cl_data = cl_data.reset_index(drop=True)
    cl_data = cl_data.drop(cl_data.index[0])
    cl_data.columns.name = None
    cl_data = cl_data.reset_index(drop=True)
    cl_data['Day'] = np.arange(1, len(cl_data)+1) #adding 'Day' maybe confused here because is similar to index but it will be really helpful latter

    cleared_df[key] = cl_data



In [111]:
cleared_df[list(dataframes.keys())[0]]

,time(UTC),T2m,RH,G(h),Gb(n),Gd(h),IR(h),WS10m,WD10m,SP,Day
0,20060101:0000,-2.96,89.35,0.0,-0.0,0.0,265.03,4.07,190.0,96549.0,1
1,20060101:0100,-2.92,89.38,0.0,-0.0,0.0,264.43,4.19,184.0,96549.0,2
2,20060101:0200,-2.88,89.41,0.0,-0.0,0.0,263.84,4.31,178.0,96549.0,3
3,20060101:0300,-2.84,89.44,0.0,-0.0,0.0,263.25,4.44,176.0,96536.0,4
4,20060101:0400,-2.8,89.48,0.0,-0.0,0.0,262.66,4.56,175.0,96524.0,5
...,...,...,...,...,...,...,...,...,...,...,...
8755,20101231:1900,-3.16,89.18,0.0,-0.0,0.0,267.98,3.45,264.0,97325.0,8756
8756,20101231:2000,-3.12,89.22,0.0,-0.0,0.0,267.39,3.58,264.0,97321.0,8757
8757,20101231:2100,-3.08,89.25,0.0,-0.0,0.0,266.8,3.7,261.0,97293.0,8758
8758,20101231:2200,-3.04,89.28,0.0,-0.0,0.0,266.21,3.82,257.0,97264.0,8759


###Upload data regarding wind turbins

In [136]:
uploaded_turbins = files.upload()

Saving Charakterystyki turbin.xlsx to Charakterystyki turbin (1).xlsx


In [138]:
for file in uploaded_turbins:
    turbine_2MW = pd.read_excel(io.BytesIO(uploaded_turbins[file]), sheet_name='2MW')
    turbine_20kW = pd.read_excel(io.BytesIO(uploaded_turbins[file]), sheet_name='20kW')

turbine_2MW.columns = ['WS80m', 'P80m']
turbine_2MW.drop(turbine_2MW.index[0], inplace=True)
turbine_2MW = turbine_2MW.astype(float)

turbine_20kW.columns = ['WS30m', 'P30m']
turbine_20kW.drop(turbine_20kW.index[0], inplace=True)
turbine_20kW = turbine_20kW.astype(float)

In [115]:
turbine_2MW.head(10)

,WS80m,P80m
1,0.0,0.0
2,0.5,0.0
3,1.0,0.0
4,1.5,0.0
5,2.0,0.0
6,2.5,0.0
7,3.0,0.0
8,3.5,42.2
9,4.0,93.3
10,4.5,145.2


In [117]:
turbine_20kW.head(10)

,WS30m,P30m
1,0,0
2,1,0
3,2,0.08
4,3,0.5
5,4,1.35
6,5,2.8
7,6,4.7
8,7,7
9,8,9.6
10,9,12.3


###Calculations


1. Assignment of variable values


    aplpha - roughness coefficient (exponent)

    h_orginal - wind measurement high [m]

    h_t1 - height of the first wind turbin [m]

    h_t2 - height of the second wind turbin [m]


In [139]:

alpha = 0.16
h_orginal = 10
h_t1 = 30
h_t2 = 80



2. Calculation of wind speed at turbine heights

In [140]:
pd.options.mode.chained_assignment = None  # default='warn', pandas is warning that my operations are not apply on my orginal frame. It's true and it's what I want here

final_results = {}

for key in cleared_df:
    cl_data = cleared_df[key][['Day','WS10m']]
    cl_data['WS30m'] = cl_data['WS10m'].apply(lambda x: float(x)*(h_t1/h_orginal)**(alpha))
    cl_data['WS80m'] = cl_data['WS10m'].apply(lambda x: float(x)*(h_t2/h_orginal)**(alpha))
    final_results[key] = cl_data
final_results[list(dataframes.keys())[0]]

,Day,WS10m,WS30m,WS80m
0,1,4.07,4.852145,5.676607
1,2,4.19,4.995206,5.843976
2,3,4.31,5.138267,6.011345
3,4,4.44,5.293250,6.192662
4,5,4.56,5.436310,6.360031
...,...,...,...,...
8755,8756,3.45,4.112998,4.811866
8756,8757,3.58,4.267981,4.993182
8757,8758,3.7,4.411041,5.160552
8758,8759,3.82,4.554102,5.327921


3. Matching the turbine power value depending on the wind speed

In [141]:
for key in final_results:

    dataP30 = pd.merge_asof(final_results[key].sort_values('WS30m'), turbine_20kW, on='WS30m')
    final_results[key] = dataP30
    dataP80 = pd.merge_asof(final_results[key].sort_values('WS80m'), turbine_2MW, on='WS80m')
    final_results[key] = dataP80.sort_values('Day') #now we see why was added 'Day' column,
    #ofc we can care about index during every change but IMO in this case it is easier to manage it in this way.


final_results[list(dataframes.keys())[0]]

,Day,WS10m,WS30m,WS80m,P30m,P80m
6922,1,4.07,4.852145,5.676607,1.35,284.2
7078,2,4.19,4.995206,5.843976,1.35,284.2
7215,3,4.31,5.138267,6.011345,2.80,390.9
7339,4,4.44,5.293250,6.192662,2.80,390.9
7452,5,4.56,5.436310,6.360031,2.80,390.9
...,...,...,...,...,...,...
6111,8756,3.45,4.112998,4.811866,1.35,145.2
6279,8757,3.58,4.267981,4.993182,1.35,145.2
6446,8758,3.7,4.411041,5.160552,1.35,211.3
6620,8759,3.82,4.554102,5.327921,1.35,211.3
